# 05. Mentor Matching Model

Сурагч-Mentor хоорондын тохирлыг тооцоолох

**Input:**
- Сурагчийн profile (сул сэдвүүд, түвшин, зорилго)
- Mentor-ийн profile (заадаг сэдвүүд, туршлага, үнэлгээ)

**Output:**
- Тохирлын оноо (0-1)
- Эрэмбэлсэн mentor жагсаалт

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
import joblib

In [ ]:
# All possible subjects/topics
ALL_SUBJECTS = [
    'math', 'physics', 'chemistry', 'biology', 'english', 'mongolian'
]

ALL_TOPICS = [
    'algebra', 'geometry', 'trigonometry', 'calculus',
    'mechanics', 'thermodynamics', 'electromagnetism',
    'organic', 'inorganic', 'physical_chemistry'
]

In [ ]:
class MentorMatcher:
    def __init__(self):
        self.subject_encoder = MultiLabelBinarizer(classes=ALL_SUBJECTS)
        self.topic_encoder = MultiLabelBinarizer(classes=ALL_TOPICS)
        self.subject_encoder.fit([ALL_SUBJECTS])
        self.topic_encoder.fit([ALL_TOPICS])
    
    def _encode_profile(self, subjects: List[str], topics: List[str], 
                        level: int = 5, rating: float = 0) -> np.ndarray:
        """Profile-г vector болгох"""
        subject_vec = self.subject_encoder.transform([subjects])[0]
        topic_vec = self.topic_encoder.transform([topics])[0]
        
        # Combine all features
        return np.concatenate([
            subject_vec,
            topic_vec,
            [level / 10],  # Normalize
            [rating / 5]   # Normalize (assuming 0-5 scale)
        ])
    
    def calculate_match_score(self, student: Dict, mentor: Dict) -> float:
        """
        Сурагч-Mentor тохирлын оноо тооцох
        
        Args:
            student: {'subjects': [...], 'weak_topics': [...], 'level': int}
            mentor: {'subjects': [...], 'topics': [...], 'rating': float}
        
        Returns:
            Тохирлын оноо (0-1)
        """
        # Encode
        student_vec = self._encode_profile(
            student.get('subjects', []),
            student.get('weak_topics', []),
            student.get('level', 5)
        )
        
        mentor_vec = self._encode_profile(
            mentor.get('subjects', []),
            mentor.get('topics', []),
            rating=mentor.get('rating', 0)
        )
        
        # Cosine similarity
        similarity = cosine_similarity([student_vec], [mentor_vec])[0][0]
        
        # Boost by rating
        rating_boost = mentor.get('rating', 0) / 5 * 0.2  # Max 20% boost
        
        # Subject overlap bonus
        student_subjects = set(student.get('subjects', []))
        mentor_subjects = set(mentor.get('subjects', []))
        overlap = len(student_subjects & mentor_subjects) / max(len(student_subjects), 1)
        subject_bonus = overlap * 0.3  # Max 30% bonus
        
        final_score = min(1.0, similarity + rating_boost + subject_bonus)
        return final_score
    
    def rank_mentors(self, student: Dict, mentors: List[Dict]) -> List[Dict]:
        """
        Mentor-уудыг тохирлоор эрэмбэлэх
        
        Returns:
            Эрэмбэлсэн mentor жагсаалт (score-тай)
        """
        scored_mentors = []
        for mentor in mentors:
            score = self.calculate_match_score(student, mentor)
            scored_mentors.append({**mentor, 'match_score': score})
        
        return sorted(scored_mentors, key=lambda x: x['match_score'], reverse=True)

In [ ]:
# Test
matcher = MentorMatcher()

# Sample student
student = {
    'subjects': ['math', 'physics'],
    'weak_topics': ['algebra', 'mechanics'],
    'level': 5
}

# Sample mentors
mentors = [
    {'id': 1, 'name': 'Mentor A', 'subjects': ['math'], 'topics': ['algebra', 'calculus'], 'rating': 4.8},
    {'id': 2, 'name': 'Mentor B', 'subjects': ['physics'], 'topics': ['mechanics', 'thermodynamics'], 'rating': 4.5},
    {'id': 3, 'name': 'Mentor C', 'subjects': ['chemistry'], 'topics': ['organic'], 'rating': 4.9},
    {'id': 4, 'name': 'Mentor D', 'subjects': ['math', 'physics'], 'topics': ['algebra', 'mechanics'], 'rating': 4.2},
]

ranked = matcher.rank_mentors(student, mentors)

print("Ranked Mentors:")
for i, m in enumerate(ranked, 1):
    print(f"{i}. {m['name']} - Score: {m['match_score']:.3f}")

In [ ]:
# Save matcher
joblib.dump(matcher, '../trained_models/mentor_matcher.pkl')
print('Mentor matcher saved!')